# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [2]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

device = '/gpu:0'

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [5]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()


(64, 10)


D:\Python\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [14]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        init = tf.keras.initializers.VarianceScaling(scale=2.0) # Инициализатор для ядер сверточных слоев и полносвязного слоя
        self.conv1 = tf.keras.layers.Conv2D(channel_1, kernel_size=(5, 5), padding='same', activation='relu', 
                                            kernel_initializer=init) # Первый сверточный слой  5 x 5 kernels, zero-padding = 'same', relu
        self.conv2 = tf.keras.layers.Conv2D(channel_2, kernel_size=(3, 3), padding='same', activation='relu',
                                            kernel_initializer=init) # Второй сверточный слой 3 x 3 kernels, zero-padding = 'same', relu
        self.flatten = tf.keras.layers.Flatten() # Слой, разворачивающий выходные данные сверточных слоев в один вектор
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=init) # Полносвязный слой с функцией активации softmax

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x) # Пропускаем входные данные x через первый сверточный слой
        x = self.conv2(x) # Пропускаем выходные данные первого сверточного слоя через второй сверточный слой
        x = self.flatten(x) # Разворачиваем выходные данные второго сверточного слоя в один вектор
        scores = self.fc(x) # Пропускаем развернутый вектор через полносвязный слой

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [15]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [16]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    
    with tf.device(device):

        print_every = 100
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [17]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0233967304229736, Accuracy: 10.9375, Val Loss: 2.9211366176605225, Val Accuracy: 11.699999809265137
Iteration 100, Epoch 1, Loss: 2.259289503097534, Accuracy: 28.078588485717773, Val Loss: 1.9010292291641235, Val Accuracy: 36.5
Iteration 200, Epoch 1, Loss: 2.089059591293335, Accuracy: 32.02736282348633, Val Loss: 1.8815157413482666, Val Accuracy: 39.39999771118164
Iteration 300, Epoch 1, Loss: 2.0073564052581787, Accuracy: 34.058345794677734, Val Loss: 1.8760639429092407, Val Accuracy: 37.900001525878906
Iteration 400, Epoch 1, Loss: 1.9383715391159058, Accuracy: 35.76605224609375, Val Loss: 1.7064533233642578, Val Accuracy: 42.0
Iteration 500, Epoch 1, Loss: 1.8919024467468262, Accuracy: 36.89807891845703, Val Loss: 1.6237136125564575, Val Accuracy: 45.69999694824219
Iteration 600, Epoch 1, Loss: 1.8597824573516846, Accuracy: 37.80418014526367, Val Loss: 1.6495980024337769, Val Accuracy: 44.400001525878906
Iteration 700, Epoch 1, Loss: 1.8334158658981323

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [18]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    input_shape = (32, 32, 3)
    channel_1, channel_2, num_classes = 32, 16, 10
    inputs = tf.keras.Input(shape=input_shape)
    
    # Сверточные и пуллинг слои
    x = tf.keras.layers.Conv2D(filters=channel_1, kernel_size=(5, 5), padding='same', activation='relu')(inputs)
    x = tf.keras.layers.Conv2D(filters=channel_2, kernel_size=(3, 3), padding='same', activation='relu')(x)
    
    # Переход к полносвязным слоям
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    learning_rate = 3e-3
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True) #Nesterov momentum = 0.9

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3675942420959473, Accuracy: 9.375, Val Loss: 2.3288252353668213, Val Accuracy: 11.0
Iteration 100, Epoch 1, Loss: 1.8450183868408203, Accuracy: 33.555076599121094, Val Loss: 1.603040337562561, Val Accuracy: 44.60000228881836
Iteration 200, Epoch 1, Loss: 1.7009004354476929, Accuracy: 39.412315368652344, Val Loss: 1.447537899017334, Val Accuracy: 49.5
Iteration 300, Epoch 1, Loss: 1.6219552755355835, Accuracy: 42.32246398925781, Val Loss: 1.3955825567245483, Val Accuracy: 52.499996185302734
Iteration 400, Epoch 1, Loss: 1.5590249300003052, Accuracy: 44.62281799316406, Val Loss: 1.3745099306106567, Val Accuracy: 51.70000076293945
Iteration 500, Epoch 1, Loss: 1.515647530555725, Accuracy: 46.13273620605469, Val Loss: 1.3159633874893188, Val Accuracy: 52.79999923706055
Iteration 600, Epoch 1, Loss: 1.488855242729187, Accuracy: 47.215576171875, Val Loss: 1.3078023195266724, Val Accuracy: 55.0
Iteration 700, Epoch 1, Loss: 1.4631694555282593, Accuracy: 48.089782

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [19]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1879892349243164, Accuracy: 7.8125, Val Loss: 3.112488269805908, Val Accuracy: 13.799999237060547
Iteration 100, Epoch 1, Loss: 2.242853879928589, Accuracy: 28.62004852294922, Val Loss: 1.874886393547058, Val Accuracy: 39.20000076293945
Iteration 200, Epoch 1, Loss: 2.0718905925750732, Accuracy: 32.532649993896484, Val Loss: 1.8591254949569702, Val Accuracy: 40.29999923706055
Iteration 300, Epoch 1, Loss: 2.0003509521484375, Accuracy: 34.35942840576172, Val Loss: 1.8574414253234863, Val Accuracy: 38.900001525878906
Iteration 400, Epoch 1, Loss: 1.9332118034362793, Accuracy: 36.21025848388672, Val Loss: 1.7265125513076782, Val Accuracy: 41.80000305175781
Iteration 500, Epoch 1, Loss: 1.8892593383789062, Accuracy: 37.259857177734375, Val Loss: 1.6562516689300537, Val Accuracy: 43.20000076293945
Iteration 600, Epoch 1, Loss: 1.8585249185562134, Accuracy: 38.18375778198242, Val Loss: 1.6782445907592773, Val Accuracy: 44.10000228881836
Iteration 700, Epoch 1, L

Альтернативный менее гибкий способ обучения:

In [20]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)



313/313 [==============================] - 1s 4ms/step - loss: 1.7165 - sparse_categorical_accuracy: 0.4134


[1.7165124416351318, 0.41339999437332153]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [21]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    channel_1, channel_2, num_classes = 32, 16, 10
    model = tf.keras.Sequential([  #Sequential
        # Сверточный слой 1
        tf.keras.layers.Conv2D(filters=channel_1, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)),
        # Сверточный слой 2
        tf.keras.layers.Conv2D(filters=channel_2, kernel_size=(3, 3), padding='same', activation='relu'),
        # Переход к полносвязному слою
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    learning_rate = 3e-3
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.311187982559204, Accuracy: 10.9375, Val Loss: 2.326413154602051, Val Accuracy: 9.899999618530273
Iteration 100, Epoch 1, Loss: 1.884738564491272, Accuracy: 32.98267364501953, Val Loss: 1.6599488258361816, Val Accuracy: 43.29999923706055
Iteration 200, Epoch 1, Loss: 1.7337342500686646, Accuracy: 39.210201263427734, Val Loss: 1.455155849456787, Val Accuracy: 49.29999923706055
Iteration 300, Epoch 1, Loss: 1.6500146389007568, Accuracy: 42.016197204589844, Val Loss: 1.4143625497817993, Val Accuracy: 49.900001525878906
Iteration 400, Epoch 1, Loss: 1.5832622051239014, Accuracy: 44.264339447021484, Val Loss: 1.375757098197937, Val Accuracy: 52.10000228881836
Iteration 500, Epoch 1, Loss: 1.5356769561767578, Accuracy: 45.780311584472656, Val Loss: 1.3055254220962524, Val Accuracy: 53.500003814697266
Iteration 600, Epoch 1, Loss: 1.5060149431228638, Accuracy: 46.81520462036133, Val Loss: 1.2726283073425293, Val Accuracy: 54.29999923706055
Iteration 700, Epoch 1, 

In [22]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)


313/313 [==============================] - 1s 2ms/step - loss: 1.4449 - sparse_categorical_accuracy: 0.4920


[1.4449098110198975, 0.492000013589859]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [23]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [24]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.432112216949463, Accuracy: 14.0625, Val Loss: 2.9769957065582275, Val Accuracy: 13.199999809265137
Iteration 100, Epoch 1, Loss: 2.248833417892456, Accuracy: 28.46534538269043, Val Loss: 1.8698688745498657, Val Accuracy: 38.0
Iteration 200, Epoch 1, Loss: 2.0785369873046875, Accuracy: 32.48600769042969, Val Loss: 1.8814749717712402, Val Accuracy: 39.20000076293945
Iteration 300, Epoch 1, Loss: 2.003537178039551, Accuracy: 34.40095520019531, Val Loss: 1.8268736600875854, Val Accuracy: 39.39999771118164
Iteration 400, Epoch 1, Loss: 1.9353119134902954, Accuracy: 36.25312042236328, Val Loss: 1.7053803205490112, Val Accuracy: 41.5
Iteration 500, Epoch 1, Loss: 1.8898378610610962, Accuracy: 37.3409423828125, Val Loss: 1.6454205513000488, Val Accuracy: 45.69999694824219
Iteration 600, Epoch 1, Loss: 1.8586888313293457, Accuracy: 38.168155670166016, Val Loss: 1.6854610443115234, Val Accuracy: 43.0
Iteration 700, Epoch 1, Loss: 1.8328218460083008, Accuracy: 38.888

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [28]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        super(CustomConvNet, self).__init__()
         # Сверточный слой с 16 фильтрами, размером ядра 3x3, функцией активации 'relu' и входным тензором размера (32, 32, 3)
        self.conv1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3))    
        # Сверточный слой с 32 фильтрами, размером ядра 3x3, функцией активации 'relu' 
        self.conv2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu')
        # Слой максимального пуллинга с окном 2 х 2
        self.pool1 = tf.keras.layers.MaxPooling2D((2,2))
        #Дропаут с параметром 0.25
        self.dropout1 = tf.keras.layers.Dropout(0.25)
        self.conv3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu')

        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2))
        #Слой для развертывания
        self.flatten = tf.keras.layers.Flatten()
        #Плотный слой
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dropout2 = tf.keras.layers.Dropout(0.5)
    
        self.dense2 = tf.keras.layers.Dense(10, activation='sigmoid') 
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.pool1(x)
        if training:
            x = self.dropout1(x, training=training)
        x = self.conv3(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        if training:
            x = self.dropout2(x, training=training)
        x = self.dense2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 4

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)  #tf.keras.optimizers.RMSprop(learning_rate)


train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.354480743408203, Accuracy: 7.8125, Val Loss: 2.365732431411743, Val Accuracy: 7.90000057220459
Iteration 100, Epoch 1, Loss: 2.1014277935028076, Accuracy: 22.431930541992188, Val Loss: 1.8432281017303467, Val Accuracy: 35.400001525878906
Iteration 200, Epoch 1, Loss: 1.96261727809906, Accuracy: 27.72076988220215, Val Loss: 1.5889312028884888, Val Accuracy: 42.89999771118164
Iteration 300, Epoch 1, Loss: 1.8624134063720703, Accuracy: 31.717193603515625, Val Loss: 1.4873244762420654, Val Accuracy: 49.29999923706055
Iteration 400, Epoch 1, Loss: 1.7834511995315552, Accuracy: 34.49189758300781, Val Loss: 1.3725366592407227, Val Accuracy: 50.400001525878906
Iteration 500, Epoch 1, Loss: 1.7238205671310425, Accuracy: 36.84817886352539, Val Loss: 1.3561595678329468, Val Accuracy: 53.500003814697266
Iteration 600, Epoch 1, Loss: 1.683825135231018, Accuracy: 38.42034149169922, Val Loss: 1.3048871755599976, Val Accuracy: 55.29999542236328
Iteration 700, Epoch 1, Los

In [14]:
#В сети используются сверточные слои, слои максимального пуллинга, dropout для снижения вероятности переобучения, 
#flatten для изменения размерности, а также плотные слои (Dense).
#Были проведены эксперименты с различными параметрами:
#Оптимизаторы RMSprop и Adam дают примерно одинаковые результаты. Небольшое преимущество за вторым вариантом.
#Было проведено сравнение функций активации relu и sigmoid: перввый вариант оказался намного эффективнее во временных затратах и по параметру accuracy.
#При использовании на выходе softmax и sigmoid не показали существенного различия в полученной точности.
#Было уменьшено число эпох, поскольку для достижения точности в 70% с использованием построенной сети их требуется намного меньше.